In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import glob,os,sys

In [2]:
obsdf = pd.read_csv('GSE166635_obs.csv',index_col=0,header=0)
obsdf

,sample,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden_0.4,cell type
HCC1_AAACCCAAGAAGAACG-1,HCC1,1729,1729,5173.0,413.0,7.983762,3,TAMs
HCC1_AAACCCAAGCAACAAT-1,HCC1,3782,3782,13257.0,905.0,6.826582,9,TAMs
HCC1_AAACCCAAGCATCTTG-1,HCC1,2750,2750,9418.0,721.0,7.655553,3,TAMs
HCC1_AAACCCAAGCTGAGTG-1,HCC1,1448,1447,4959.0,257.0,5.182497,0,HPCs
HCC1_AAACCCACAAGGGCAT-1,HCC1,2080,2080,8340.0,314.0,3.764988,0,HPCs
...,...,...,...,...,...,...,...,...
HCC2_TTTGTTGCAATACGCT-1,HCC2,487,487,824.0,52.0,6.310680,4,T cells
HCC2_TTTGTTGCACATACGT-1,HCC2,890,890,1929.0,130.0,6.739243,4,T cells
HCC2_TTTGTTGGTTAGAAGT-1,HCC2,1060,1060,2930.0,169.0,5.767918,4,T cells
HCC2_TTTGTTGTCGCCTAGG-1,HCC2,677,677,1832.0,138.0,7.532751,10,B cells


In [3]:
set(obsdf['sample'])

{'HCC1', 'HCC2'}

In [4]:
splMap = {
'SRR13691898':'HCC1',
'SRR13691899':'HCC2',
}

In [5]:
spls = []
for fn in splMap:
    genusdf = pd.read_csv('/data/hour/nb/Lip/PRJNA701452_invadeseq/'+fn+'_nova.filtered_matrix.genus.csv',index_col=0,header=0)
    #spl = fn #os.path.basename(fn)
    print('='*10)
    print(fn)
    oldPat = fn+'_'
    newPat = splMap[fn]+'_'
    print(oldPat,newPat)
    obsidx = [i for i in obsdf.index if newPat in i]
    newidx = [newPat+i.split('_')[1] for i in genusdf.index]
    genusdf.index = newidx
    comCells = set(newidx).intersection(set(obsidx))
    print('invadeseq: %s, common: %s' % (len(genusdf),len(comCells)))
    genusdf = genusdf.reindex(obsidx, fill_value=0)
    print(genusdf.shape)
    spls.append(genusdf)
newobs = pd.concat(spls).fillna(0)
print(newobs.shape)

SRR13691898
SRR13691898_ HCC1_
invadeseq: 2323, common: 2136
(14855, 12)
SRR13691899
SRR13691899_ HCC2_
invadeseq: 120, common: 101
(7486, 20)
(22341, 26)


In [6]:
newobs = newobs.groupby(newobs.index).sum()#去重
newobs

,Achromobacter,Acinetobacter,Burkholderia,Citrobacter,Corynebacterium,Leptotrichia,Malassezia,Orthohepadnavirus,Plasmodium,Pseudomonas,...,Exiguobacterium,Flavobacterium,Hymenobacter,Mycoplasma,Nonlabens,Paucibacter,Phytophthora,Psychrobacter,Pythium,Shewanella
HCC1_AAACCCAAGAAGAACG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCAACAAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCATCTTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCTGAGTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCACAAGGGCAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC2_TTTGTTGCAATACGCT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGCACATACGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGGTTAGAAGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGTCGCCTAGG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
newobs = newobs.loc[:,newobs.max()>0]#过滤
newobs

,Achromobacter,Acinetobacter,Burkholderia,Citrobacter,Corynebacterium,Leptotrichia,Malassezia,Orthohepadnavirus,Plasmodium,Pseudomonas,...,Exiguobacterium,Flavobacterium,Hymenobacter,Mycoplasma,Nonlabens,Paucibacter,Phytophthora,Psychrobacter,Pythium,Shewanella
HCC1_AAACCCAAGAAGAACG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCAACAAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCATCTTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCTGAGTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCACAAGGGCAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC2_TTTGTTGCAATACGCT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGCACATACGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGGTTAGAAGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGTCGCCTAGG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
newobs = newobs.reindex(obsdf.index, fill_value=0)#补齐
newobs

,Achromobacter,Acinetobacter,Burkholderia,Citrobacter,Corynebacterium,Leptotrichia,Malassezia,Orthohepadnavirus,Plasmodium,Pseudomonas,...,Exiguobacterium,Flavobacterium,Hymenobacter,Mycoplasma,Nonlabens,Paucibacter,Phytophthora,Psychrobacter,Pythium,Shewanella
HCC1_AAACCCAAGAAGAACG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCAACAAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCATCTTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCAAGCTGAGTG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1_AAACCCACAAGGGCAT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC2_TTTGTTGCAATACGCT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGCACATACGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGGTTAGAAGT-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC2_TTTGTTGTCGCCTAGG-1,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
newobs.to_csv('GSE166635_invadeseq.csv',index=True,header=True)